<a href="https://colab.research.google.com/github/gustavokubiack/ciencia-de-dados/blob/main/Tecnicas_Amostragem_Tarefa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Amostragem

**Amostragem é uma contagem ou medição de parte da população e é usada em estudos estatísticos**

Observação: Para coletar dados não tendenciosos, você
deve ter certeza de que a amostra represente a população

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Carregamento da base de dados

In [2]:
import pandas as pd
import random
import numpy as np

In [4]:
dataset = pd.read_csv('/content/drive/MyDrive/ciencia_de_dados/credit_data.csv')

In [5]:
dataset.shape

(2000, 5)

In [6]:
dataset.head()

,i#clientid,income,age,loan,c#default
0,1,66155.925095,59.017015,8106.532131,0
1,2,34415.153966,48.117153,6564.745018,0
2,3,57317.170063,63.108049,8020.953296,0
3,4,42709.534201,45.751972,6103.642260,0
4,5,66952.688845,18.584336,8770.099235,1


In [7]:
dataset.tail()

,i#clientid,income,age,loan,c#default
1995,1996,59221.044874,48.518179,1926.729397,0
1996,1997,69516.127573,23.162104,3503.176156,0
1997,1998,44311.449262,28.017167,5522.786693,1
1998,1999,43756.056605,63.971796,1622.722598,0
1999,2000,69436.579552,56.152617,7378.833599,0


## Amostragem aleatória simples

In [10]:
def amostragem_aleatoria_simples(dataset, amostras):
  return dataset.sample(n = amostras, random_state=1)

In [11]:
df_amostra_aleatoria_simples = amostragem_aleatoria_simples(dataset, 1000)
df_amostra_aleatoria_simples.shape

(1000, 5)

## Amostragem sistemática

In [14]:
def amostragem_sistematica(dataset, amostras):
  intervalo = len(dataset) // amostras
  random.seed(1)
  inicio = random.randint(0, intervalo)
  indices = np.arange(inicio, len(dataset), step = intervalo)
  amostra_sistematica = dataset.iloc[indices]
  return amostra_sistematica

In [15]:
df_amostra_sistematica = amostragem_sistematica(dataset, 1000)
df_amostra_sistematica.shape

(1000, 5)

## Amostragem por grupos

In [16]:
def amostragem_agrupamento(dataset, numero_grupos):
  intervalo = len(dataset) / numero_grupos

  grupos = []
  id_grupo = 0
  contagem = 0
  for _ in dataset.iterrows():
    grupos.append(id_grupo)
    contagem += 1
    if contagem > intervalo:
      contagem = 0
      id_grupo += 1

  dataset['grupo'] = grupos
  random.seed(1)
  grupo_selecionado = random.randint(0, numero_grupos)
  return dataset[dataset['grupo'] == grupo_selecionado]

In [17]:
len(dataset) / 2

1000.0

In [18]:
df_amostra_agrupamento = amostragem_agrupamento(dataset, 2)
df_amostra_agrupamento.shape, df_amostra_agrupamento['grupo'].value_counts()

((1001, 6),
 grupo
 0    1001
 Name: count, dtype: int64)

## Amostra estratificada

In [19]:
from sklearn.model_selection import StratifiedShuffleSplit

In [20]:
dataset['c#default'].value_counts()

,count
c#default,
0,1717
1,283


In [21]:
def amostragem_estratificada(dataset, percentual, campo):
  split = StratifiedShuffleSplit(test_size=percentual, random_state=1)
  for _, y in split.split(dataset, dataset[campo]):
    df_y = dataset.iloc[y]
  return df_y

In [22]:
df_amostra_estratificada = amostragem_estratificada(dataset, 0.5, 'c#default')
df_amostra_estratificada.shape

(1000, 6)

## Comparativo dos resultados

In [23]:
dataset['age'].mean(), dataset['income'].mean(), dataset['loan'].mean()

(np.float64(40.80755937840458),
 np.float64(45331.600017793244),
 np.float64(4444.369694688258))

In [24]:
df_amostra_aleatoria_simples['age'].mean(), df_amostra_aleatoria_simples['income'].mean(), df_amostra_aleatoria_simples['loan'].mean()

(np.float64(40.49552561124429),
 np.float64(45563.268653769046),
 np.float64(4449.446900442365))

In [25]:
df_amostra_sistematica['age'].mean(), df_amostra_sistematica['income'].mean(), df_amostra_sistematica['loan'].mean()

(np.float64(40.91117381141754),
 np.float64(45691.49875066942),
 np.float64(4506.78797642633))

In [26]:
df_amostra_agrupamento['age'].mean(), df_amostra_agrupamento['income'].mean(), df_amostra_agrupamento['loan'].mean()

(np.float64(41.0432231120503),
 np.float64(44846.74925986141),
 np.float64(4390.161493744205))

In [27]:
df_amostra_estratificada['age'].mean(), df_amostra_estratificada['income'].mean(), df_amostra_estratificada['loan'].mean()

(np.float64(40.53363707173919),
 np.float64(45101.366750267334),
 np.float64(4423.7446510941945))

**Dica: faça a subtração dos valores dos atributos ("age", "income", "loan") do dataset original com os dos atributos de cada tipo de amostragem. A técnica a ser utilizada por você é aquela que tiver as menores diferenças.**

In [30]:
import pandas as pd

media_original = {
    "idade": dataset['age'].mean(),
    "renda": dataset['income'].mean(),
    "emprestimo": dataset['loan'].mean()
}

amostras_dfs = {
    "amostra_aleatoria": df_amostra_aleatoria_simples,
    "amostra_sistematica": df_amostra_sistematica,
    "amostra_agrupamento": df_amostra_agrupamento,
    "amostra_estratificada": df_amostra_estratificada
}

resultados_comparacao = []

for nome_amostra, df_amostra in amostras_dfs.items():
    diferencas = {
        "amostra": nome_amostra,
        "idade_dif": abs(media_original["idade"] - df_amostra['age'].mean()),
        "renda_dif": abs(media_original["renda"] - df_amostra['income'].mean()),
        "emprestimo_dif": abs(media_original["emprestimo"] - df_amostra['loan'].mean())
    }
    resultados_comparacao.append(diferencas)

df_comparacao = pd.DataFrame(resultados_comparacao)

print("Diferenças absolutas em relação ao dataset original:\n")
print(df_comparacao)

melhor_amostra_por_atributo = {
    "idade": df_comparacao.loc[df_comparacao['idade_dif'].idxmin(), "amostra"],
    "renda": df_comparacao.loc[df_comparacao['renda_dif'].idxmin(), "amostra"],
    "emprestimo": df_comparacao.loc[df_comparacao['emprestimo_dif'].idxmin(), "amostra"],
}

print("\nMelhor técnica por atributo:")
for atributo, tecnica in melhor_amostra_por_atributo.items():
    print(f"- {atributo}: {tecnica}")


Diferenças absolutas em relação ao dataset original:

                 amostra  idade_dif   renda_dif  emprestimo_dif
0      amostra_aleatoria   0.312034  231.668636        5.077206
1    amostra_sistematica   0.103614  359.898733       62.418282
2    amostra_agrupamento   0.235664  484.850758       54.208201
3  amostra_estratificada   0.273922  230.233268       20.625044

Melhor técnica por atributo:
- idade: amostra_sistematica
- renda: amostra_estratificada
- emprestimo: amostra_aleatoria
